In [4]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

from nltk import word_tokenize, WordNetLemmatizer
from nltk.corpus import stopwords

Using TensorFlow backend.
/Users/thomasvanorden/anaconda3/envs/NTMI/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/thomasvanorden/anaconda3/envs/NTMI/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/thomasvanorden/anaconda3/envs/NTMI/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [11]:
def txt_to_lists(directory):
    data = []
    for file in os.listdir(directory):
        f = open(directory+'/'+file)
        data.append(f.readlines()[0])
    
    return data


def create_data_dict(positive, negative):
    data_dict = {
        "text" : positive + negative,
        "label" : [1 for i in range(len(positive))]
        + [0 for i in range(len(negative))]
    }
    
    return data_dict


def extract_and_save(directory, filename):
    print("Creating positive and negative bundles...")
    train_pos_data = txt_to_lists(directory+"/pos")
    train_neg_data = txt_to_lists(directory+"/neg")
    
    print("Combining bundles into one dictionary with labels...")
    train_data_dict = create_data_dict(train_pos_data, train_neg_data)
    
    print("Saving dictionary as pandas dataframe...")
    df = pd.DataFrame(train_data_dict)
    df.to_csv(filename,index=False)
    
    print(f"Saved as `{filename}`.")

In [12]:
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)

    return text_nopunct


def lower_token(tokens): 
    return [w.lower() for w in tokens]


def remove_stop_words(tokens, language='english'):
    stoplist = stopwords.words(language)
    
    filtered = [word for word in tokens if word not in stoplist]
    joined = [' '.join(sen) for sen in filtered]

    return filtered, joined


def tokenize(data, lower=True):
    tokens = []
    
    for sen in data:
        tokenized_sen = word_tokenize(sen)
        if lower:
            tokenized_sen = [word.lower() for word in tokenized_sen]
        
        tokens.append(tokenized_sen)
        
    return tokens


def clean_and_save(file_read, file_write):
    print(f"Reading data from `{file_read}` and removing punctiation...")
    data = pd.read_csv(file_read)
    data['text'] = data['text'].apply(lambda x: remove_punct(x))
    
    print("Tokenizing data...")
    tokens = tokenize(data.text)
    filtered_tokens, joined_tokens = remove_stop_words(tokens)
    
    print("Creating dataframe with joined and normal tokens...")
    data['text_final'] = joined_tokens
    data['tokens'] = filtered_tokens
    
    print("Saving dataframe...")
    data = data[['text_final', 'tokens', 'label']]
    
    !mkdir -p cleaned_data
    
    data.to_csv("cleaned_data/"+file_write,index=False)
    
    print(f"Saved as `{file_write}`.")
    
    return data

In [13]:
# Change train_test to 'test' to extract, clean and save the test data
train_test = "test" # "test"

In [14]:
filename = f"raw_{train_test}_data.csv"
extract_and_save(train_test, filename)

Creating positive and negative bundles...
Combining bundles into one dictionary with labels...
Saving dictionary as pandas dataframe...
Saved as `raw_test_data.csv`.


In [15]:
file_write = f"cleaned_{train_test}_data.csv"
cleaned_data = clean_and_save(filename, file_write)

Reading data from `raw_test_data.csv` and removing punctiation...
Tokenizing data...
Creating dataframe with joined and normal tokens...
Saving dataframe...
Saved as `cleaned_test_data.csv`.


In [18]:
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [19]:
from nltk.stem import PorterStemmer
from IPython.display import clear_output
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag

# Create a stemmer
ps = PorterStemmer()

# Create a lemmer
lem = WordNetLemmatizer()

length = len(cleaned_data)
stemmed_sents = []
lemmed_sents = []

for index, row in cleaned_data.iterrows():
    # Update progress
    clear_output()
    print(f'{(index/length)*100}%')
    
    tagged_sent = pos_tag(row['tokens'])
    lemmed_sent, stemmed_sent = [], []
    
    for word, tag in tagged_sent:
        # Stem the word
        stemmed_sent.append(ps.stem(word))
        
        # Lemmatize the word
        tag = get_wordnet_pos(tag)
        lemmed_sent.append(lem.lemmatize(word, pos=tag))

    # Add lemmed and stemmed sentence
    stemmed_sents.append(stemmed_sent)
    lemmed_sents.append(lemmed_sent)

cleaned_data['stemmed'] = stemmed_sents
cleaned_data['lemmed'] = lemmed_sents
cleaned_data

0.38%


KeyboardInterrupt: 

In [ ]:
# Save cleaned data with lemmatization and stemmatization
!mkdir -p cleaned_data

file_write = f"cleaned_data_{train_test}_with_lem_stem"
cleaned_data.to_csv(f"cleaned_data/"+file_write,index=False)

print(f"Saved as `{file_write}`.")